In [1]:
import re
import codecs

def parse_user_results(lines):
    """
    사용자(Ux) 한 블록의 라인 목록을 받아,
    다양한 패턴을 'Question {번호}: {값}' 형태로 변환해주는 함수.
    """

    user_results = []
    auto_q_number = 1  # (auto increment) 패턴에서 Question 번호가 없을 때 쓸 값

    # 1. 새롭게 추가된 패턴들 (우선순위가 높으니, 리스트 상단)
    extra_patterns = [
        # (A1) 1. [Mask] is 6.
        #   예) "1. [Mask] is 6."
        (
            r'^(\d+)\.\s*\[mask\]\s*is\s+(\d+)(?:\D|$)',
            'explicit_q'
        ),
        # (A2) [Mask is] 9
        #   예) "[Mask is] 9. Tatt for kjøring ..."
        #   -> Question 번호가 없으니 auto_q로 처리
        (
            r'^\[mask\s+is\]\s+(\d+)(?:\D|$)',
            'auto_q'
        ),
        # (A3) [Mask for Question X is] 2
        #   예) "[Mask for Question 1 is] 2"
        (
            r'^\[mask\s+for\s+question\s+(\d+)\s+is\]\s+(\d+)(?:\D|$)',
            'explicit_q'
        ),
        # (A4) [Mask for Question X] [Mask: Y]
        #   예) "[Mask for Question 1] [Mask: 10] ..."
        (
            r'^\[mask\s+for\s+question\s+(\d+)\]\s*\[mask:\s*(\d+)\](?:\D|$)',
            'explicit_q'
        ),
    ]

    # 2. 원래 추가했던 확장 패턴들 (ex: [Mask for Question X is Y], [Mask for Question X] is Y, 등)
    #    이 순서는 기존 답변에서 extra_patterns 로 만든 것.
    #    단, 위에서 이미 비슷한 패턴을 추가했으므로 중복되는지 주의
    old_extra_patterns = [
        # [Mask for Question X is Y]
        (
            r'^\[mask\s+for\s+question\s+(\d+)\s+is\s+(\d+)\]$',
            'explicit_q'
        ),
        # [Mask for Question X] is Y
        (
            r'^\[mask\s+for\s+question\s+(\d+)\]\s+is\s+(\d+)(?:\D|$)',
            'explicit_q'
        ),
        # [Mask for Question X] Y
        (
            r'^\[mask\s+for\s+question\s+(\d+)\]\s+(\d+)(?:\D|$)',
            'explicit_q'
        ),
        # (\d+)\.\s+(\d+)
        # (이미 (A1)에서 1. [Mask] is 6. 처리를 했으니, 여기서는 "1. 9" 형식만 처리할 수 있음)
        (
            r'^(\d+)\.\s+(\d+)(?:\D|$)',
            'explicit_q'
        ),
    ]

    # 3. 기존 base_patterns (A) ~ (E) (질문에 이미 있던 것)
    #    - [Mask] (for|in) Question X is Y
    #    - [Mask] for Question X) Y
    #    - (Question )?X) [Mask] is Y
    #    - [Mask] is Y (auto_q)
    #    - (Question )?X) Y
    base_patterns = [
        (
            r'^\[mask\]\s*(?:for|in)\s+question\s+(\d+)\D+is\s+(\d+)(?:\D|$)',
            'explicit_q'
        ),
        (
            r'^\[mask\]\s*for\s+question\s+(\d+)\)\s+(\d+)(?:\D|$)',
            'explicit_q'
        ),
        (
            r'^(?:question\s+)?(\d+)\)\s*\[mask\]\s*is\s+(\d+)(?:\D|$)',
            'explicit_q'
        ),
        (
            r'^\[mask\]\s*is\s+(\d+)(?:\D|$)',
            'auto_q'
        ),
        (
            r'^(?:question\s+)?(\d+)\)\s*(\d+)(?:\D|$)',
            'explicit_q'
        ),
    ]

    # 최종 패턴 리스트
    patterns = (
        extra_patterns
        + old_extra_patterns
        + base_patterns
    )

    for line in lines:
        line = line.strip()
        if not line:
            continue

        matched = False
        question_num = None
        answer_num   = None

        # 여러 패턴 중 "처음으로" 매칭되는 것만 사용
        for pattern, pattern_type in patterns:
            m = re.search(pattern, line, flags=re.IGNORECASE)
            if m:
                matched = True
                if pattern_type == 'explicit_q':
                    # 그룹(1) = question, 그룹(2) = answer
                    question_num = m.group(1)
                    answer_num   = m.group(2)
                elif pattern_type == 'auto_q':
                    # 그룹(1) = answer, question_num = auto_q_number
                    question_num = str(auto_q_number)
                    answer_num   = m.group(1)
                    auto_q_number += 1
                break

        # 매칭된 패턴이 있으면 결과에 추가
        if matched and question_num and answer_num:
            user_results.append(f"Question {question_num}: {answer_num}")

    return user_results


def standardize_results(input_filename, output_filename):
    """
    1) input_filename (UTF-8 or UTF-8-SIG) 파일 읽기
    2) [Uxx] 블록별로 parse_user_results() 적용
    3) output_filename에 결과 쓰기
    """
    input_path = f'../../results/gpt_result/{input_filename}'
    output_path = f'../../results/gpt_result/{output_filename}'

    # BOM 제거를 위해 utf-8-sig 사용 (BOM 없으면 그냥 UTF-8로 처리됨)
    with codecs.open(input_path, 'r', 'utf-8-sig') as f:
        lines = f.readlines()

    user_data = {}
    current_user = None
    current_lines = []

    for line in lines:
        line_stripped = line.strip()
        if not line_stripped:
            continue

        # 사용자 식별자 [U1], [U25], [U64], [U95], ...
        user_match = re.match(r'^\[U(\d+)\]', line_stripped, flags=re.IGNORECASE)
        if user_match:
            # 이전 사용자 처리
            if current_user is not None and current_lines:
                user_data[current_user] = parse_user_results(current_lines)
                current_lines = []

            current_user = "U" + user_match.group(1)
        else:
            # 이미 사용자 블록이 시작되었다면, 해당 라인을 누적
            if current_user:
                current_lines.append(line_stripped)
            # 아직 [Uxx] 전이면 스킵
            else:
                continue

    # 마지막 사용자 처리
    if current_user and current_lines:
        user_data[current_user] = parse_user_results(current_lines)

    # 결과 쓰기
    with open(output_path, 'w', encoding='utf-8') as outfile:
        for user, results in user_data.items():
            outfile.write(f"[{user}]\n")
            if results:
                for r in results:
                    outfile.write(r + "\n")
            else:
                outfile.write("No valid lines found\n")
            outfile.write("\n")


In [ ]:
input_filename = '[250304] 15015_15057_negative.txt'
output_filename = '[250304] 15015_15057_negative3.txt'
standardize_results(input_filename, output_filename)


: 